<a href="https://colab.research.google.com/github/trippingtrips/apirapid/blob/master/032824_Living_Income_Sustainable_yield_and_land.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **DATA PREPARATION**

In [46]:
#Dependencies
from google.colab import files
import io
import pandas as pd
import numpy as np
import pandas as pd
from tabulate import tabulate

In [48]:
#UPLOAD CSV FILE downloaded from looker https://enveritas.looker.com/explore/pipeline_analytics/coffee_cross_country_ops_data?qid=7Krx6dtnqcxoKd8jArODVg
def ingest_data():
    csv_file = files.upload()
    fname = list(csv_file.keys())[0]
    df = pd.read_csv(io.BytesIO(csv_file[fname]))
    return df

df = ingest_data()
print("\n*******\n\nSuccessfully loaded file!")


Saving final_data0204.csv to final_data0204 (3).csv

*******

Successfully loaded file!


In [49]:
print(df.head)

<bound method NDFrame.head of       Common Use Fields Harvest Year Common Use Fields Country Name*  \
0                          2022-2023                        Colombia   
1                          2022-2023                        Colombia   
2                          2022-2023                        Colombia   
3                          2022-2023                        Colombia   
4                          2022-2023                        Colombia   
...                              ...                             ...   
42050                      2022-2023                          Uganda   
42051                      2022-2023                          Uganda   
42052                      2022-2023                          Uganda   
42053                      2022-2023                          Uganda   
42054                      2022-2023                          Uganda   

      Common Use Fields Producing Area* Common Use Fields Supply Unit*  \
0                       co_pa_a

In [50]:
# Ethiopia for JDE is divided into two, garden and forest coffee, we will add a new column with the respective type, it will be added to all the data set but only used for Ethiopia
def determine_shade_type(canopy):
    if canopy == "heavy_shade_61_100_percent":
        return "forest"
    else:
        return "garden"

# Adding the column applying the def
df['Shade'] = df['Survey Data Obs canopy'].apply(determine_shade_type)


In [51]:
# This will change the name of the columns
df.columns = [
    'Harvest Year',
    'Country Name',
    'Producing Area',
    'Supply Unit',
    'Meta Form ID',
    'QI1 (Scoring Input) coffee type grown',
    'Survey Data Main coffee type',
    'Obs canopy',
    'Median yield (kg/ha) of green',
    'Median coffee farm size (ha)',
    'AP1 average',
    'AP2 average',
    'Shade'
]


These are the new columns added to the df, needed for the analysis:

In [52]:
# Dictionaries

countries = [
    'Indonesia',
    'Nicaragua',
    'Peru',
    'Ethiopia (garden)',
    'Ethiopia (forest)',
    'Colombia',
    'Honduras',
    'Uganda (arabica)',
    'Uganda (robusta)'
]

sustainable_yield_conditions = {
    'Indonesia': (1000, 'green'),
    'Nicaragua': (1180, 'dry'),
    'Peru': (1380, 'dry'),
    'Colombia': (1800, 'green'),
    'Honduras': (1500, 'dry'),
    'Uganda (arabica)': (640, 'dry'),
    'Uganda (robusta)': (1000, 'dry'),
    'Ethiopia (garden)': (550, 'green'),
    'Ethiopia (forest)': (800, 'green')
}

viable_farm_size_conditions = {
    'Indonesia': 1,
    'Nicaragua': 2.1,
    'Peru': 3.5,
    'Colombia': 2.8,
    'Honduras': 3,
    'Uganda (arabica)': 2,
    'Uganda (robusta)': 3,
    'Ethiopia (garden)': 1.4,
    'Ethiopia (forest)': 2.5
}

In [53]:
# Main coffee type
df['Main coffee type'] = df.apply(lambda x: x['Survey Data Main coffee type']
                                  if pd.isnull(x['QI1 (Scoring Input) coffee type grown'])
                                  else x['QI1 (Scoring Input) coffee type grown'], axis=1)

def define_country_final(row):
    # For Ethiopia, merge country name with 'Shade' in parentheses
    if row['Country Name'] == 'Ethiopia':
        return f"{row['Country Name']} ({row['Shade']})"
    # For Uganda, check if 'Main coffee type' is 'both'
    elif row['Country Name'] == 'Uganda':
        if row['Main coffee type'] == 'both':
            # Use 'Survey Data Main coffee type' when 'Main coffee type' is 'both'
            return f"{row['Country Name']} ({row['Survey Data Main coffee type']})"
        else:
            # Use 'Main coffee type' directly when it's not 'both'
            return f"{row['Country Name']} ({row['Main coffee type']})"
    # For other countries, use the country name as is
    else:
        return row['Country Name']

# Apply the function to each row
df['country final'] = df.apply(define_country_final, axis=1)



In [54]:
print(df.head)

<bound method NDFrame.head of       Harvest Year Country Name   Producing Area              Supply Unit  \
0        2022-2023     Colombia  co_pa_antioquia  co_su_antioquia_central   
1        2022-2023     Colombia  co_pa_antioquia  co_su_antioquia_central   
2        2022-2023     Colombia  co_pa_antioquia  co_su_antioquia_central   
3        2022-2023     Colombia  co_pa_antioquia  co_su_antioquia_central   
4        2022-2023     Colombia  co_pa_antioquia  co_su_antioquia_central   
...            ...          ...              ...                      ...   
42050    2022-2023       Uganda  ug_pa_west_nile         ug_su_zombo_west   
42051    2022-2023       Uganda  ug_pa_west_nile         ug_su_zombo_west   
42052    2022-2023       Uganda  ug_pa_west_nile         ug_su_zombo_west   
42053    2022-2023       Uganda  ug_pa_west_nile         ug_su_zombo_west   
42054    2022-2023       Uganda  ug_pa_west_nile         ug_su_zombo_west   

                               Meta Form ID  

In [55]:
# Median yield (kg/ha) of dry parchment
df['Median yield (kg/ha) of dry parchment'] = df['Median yield (kg/ha) of green'] * 0.8

# Meets sustainable yield
def meets_sustainable_yield(row):
    country_final = row['country final']  # Use the final country name
    yield_type = 'dry' if 'dry' in sustainable_yield_conditions.get(country_final, ('', ''))[1] else 'green'
    yield_value = row['Median yield (kg/ha) of dry parchment'] if yield_type == 'dry' else row['Median yield (kg/ha) of green']
    required_yield, required_type = sustainable_yield_conditions.get(country_final, (0, ''))

    return 1 if yield_type == required_type and yield_value >= required_yield else 0

df['Meets sustainable yield'] = df.apply(meets_sustainable_yield, axis=1)

In [56]:
# Has viable farm size
def has_viable_farm_size(row):
    country_final = row['country final']  # Use the final country name
    farm_size = row['Median coffee farm size (ha)']
    required_size = viable_farm_size_conditions.get(country_final, 0)

    return 1 if farm_size >= required_size else 0

df['Has viable farm size'] = df.apply(has_viable_farm_size, axis=1)

In [57]:
# Meets sustainable yields and viable farm size
df['Meets sustainable yields and viable farm size'] = df.apply(lambda x: 1 if x['Meets sustainable yield'] == 1
                                                               and x['Has viable farm size'] == 1 else 0, axis=1)

# Neither meets sustainable yield and viable farm size
df['Neither meets sustainable yield and viable farm size'] = df.apply(lambda x: 1 if x['Meets sustainable yield'] == 0
                                                                      and x['Has viable farm size'] == 0 else 0, axis=1)

# Below extreme poverty line
df['Below extreme poverty line'] = 1 - df['AP1 average']

# Below poverty line
df['Below poverty line'] = 1 - df['AP2 average']


In [58]:

# Display the first few rows to verify the results
print(df.head)


<bound method NDFrame.head of       Harvest Year Country Name   Producing Area              Supply Unit  \
0        2022-2023     Colombia  co_pa_antioquia  co_su_antioquia_central   
1        2022-2023     Colombia  co_pa_antioquia  co_su_antioquia_central   
2        2022-2023     Colombia  co_pa_antioquia  co_su_antioquia_central   
3        2022-2023     Colombia  co_pa_antioquia  co_su_antioquia_central   
4        2022-2023     Colombia  co_pa_antioquia  co_su_antioquia_central   
...            ...          ...              ...                      ...   
42050    2022-2023       Uganda  ug_pa_west_nile         ug_su_zombo_west   
42051    2022-2023       Uganda  ug_pa_west_nile         ug_su_zombo_west   
42052    2022-2023       Uganda  ug_pa_west_nile         ug_su_zombo_west   
42053    2022-2023       Uganda  ug_pa_west_nile         ug_su_zombo_west   
42054    2022-2023       Uganda  ug_pa_west_nile         ug_su_zombo_west   

                               Meta Form ID  

In [59]:
# Save the DataFrame to a CSV file (only if you want to check it, to download go to the side panel>Files)
df.to_csv('modified_data_0504_2.csv', index=False)


## **CPM results + results SU**

In [68]:
#UPLOAD CSV FILE downloaded from looker https://enveritas.looker.com/explore/sini/purchase?qid=1jCAr0nVWAEs0JvoF5BAMZ
def ingest_data():
    csv_file = files.upload()
    fname = list(csv_file.keys())[0]
    df = pd.read_csv(io.BytesIO(csv_file[fname]))
    return df

df_cpm = ingest_data()
print("\n*******\n\nSuccessfully loaded file!")


Saving sini_final0204_1.csv to sini_final0204_1 (1).csv

*******

Successfully loaded file!


In [69]:
#This will change the name of the columns
df_cpm.columns = [
    'Harvest Filters Country',
    'Supply Units Supply Unit Name',
    'PA code',
    'PA name',
    'Coffee Purchases Sum Purchase Volume (Kg)'
]


df_cpm['SU alias'] = df_cpm['Supply Units Supply Unit Name'] #I'm unsure of the SU alias column, where does it comes from, so for now I am just duplicating
df_cpm['Harvest year'] = "2022-2023"

In [70]:
def add_average_column(source_df, target_df, source_col, target_col, su_alias_col='SU alias', harvest_year_col='Harvest year'):
    # Create a temporary DataFrame grouped by Supply Unit and Harvest Year with the mean of the specified column
    temp_df = source_df.groupby(['Supply Unit', 'Harvest Year'])[source_col].mean().reset_index()

    # Rename columns for merging
    temp_df.rename(columns={'Supply Unit': su_alias_col, 'Harvest Year': harvest_year_col, source_col: target_col}, inplace=True)

    # Merge the calculated averages into the target DataFrame based on SU alias and Harvest Year
    merged_df = pd.merge(target_df, temp_df, how='left', on=[su_alias_col, harvest_year_col])

    # Fill NaN values with a placeholder or leave as is for no matches
    #merged_df[target_col].fillna(np.nan, inplace=True)

    return merged_df


In [71]:
# Add '% farmers meeting sustainable yields and viable farm size'
df_cpm = add_average_column(df, df_cpm, 'Meets sustainable yields and viable farm size', '% farmers meeting sustainable yields and viable farm size')

# Add '% farmers meeting sustainable yields'
df_cpm = add_average_column(df, df_cpm, 'Meets sustainable yield', '% farmers meeting sustainable yields')

# Add '% farmers having viable farm sizes'
df_cpm = add_average_column(df, df_cpm, 'Has viable farm size', '% farmers having viable farm sizes')

# Add '% farmers with neither'
df_cpm = add_average_column(df, df_cpm, 'Neither meets sustainable yield and viable farm size', '% farmers with neither')

# Add '% farmers below poverty line'
df_cpm = add_average_column(df, df_cpm, 'Below poverty line', '% farmers below poverty line')

# Add '% farmers below extreme poverty line'
df_cpm = add_average_column(df, df_cpm, 'Below extreme poverty line', '% farmers below extreme poverty line')


In [72]:
#Adding country final column
# Step 1: Create a mapping from Supply Unit to country final in df
supply_unit_to_country_final = df.set_index('Supply Unit')['country final'].to_dict()

# Step 2: Use the mapping to add "Country final" to df_cpm
df_cpm['Country final'] = df_cpm['Supply Units Supply Unit Name'].map(supply_unit_to_country_final)




In [73]:
# Get the list of all column names
columns = list(df_cpm.columns)

# Find the index of 'Harvest Filters Country' and 'Country final'
harvest_index = columns.index('Harvest Filters Country')
country_final_index = columns.index('Country final')

# Swap the positions of 'Harvest Filters Country' and 'Country final' in the list
columns[harvest_index], columns[country_final_index] = columns[country_final_index], columns[harvest_index]

# Reorder the DataFrame columns using the new list
df_cpm = df_cpm[columns]


In [74]:
print(df_cpm.head)

<bound method NDFrame.head of         Country final Supply Units Supply Unit Name  \
0    Uganda (robusta)           ug_su_bushenyi_east   
1    Uganda (arabica)              ug_su_bunyangabu   
2    Uganda (arabica)          ug_su_bulambuli_west   
3    Uganda (arabica)         ug_su_bulambuli_south   
4    Uganda (arabica)         ug_su_bulambuli_north   
..                ...                           ...   
535          Colombia     co_su_antioquia_northwest   
536               NaN         co_su_antioquia_north   
537          Colombia          co_su_antioquia_east   
538          Colombia       co_su_antioquia_central   
539          Colombia     co_su_valle_south_central   

                        PA code               PA name  \
0    ug_pa_south_western_uganda  South Western Uganda   
1          ug_pa_western_uganda        Western Uganda   
2                   ug_pa_elgon                 Elgon   
3                   ug_pa_elgon                 Elgon   
4                   ug_p

In [67]:
# Save the DataFrame to a CSV file (only if you want to check it, to download go to the side panel>Files)
df_cpm.to_csv('modified_datacpm0204.csv', index=False)